# **<center>WEB SCRAPING EN DATOS ABIERTOS COLOMBIA</center>**
**<center> ¿Qué información publican las entidades públicas sobre procesos meritocráticos? </center>**

## **LIBRERÍAS**

In [1]:
import pandas as pd
import numpy as np
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

## **WEB DRIVER**

In [2]:
#Usar Brave chromedriver
driverPath = '/usr/local/Caskroom/chromedriver/107.0.5304.62/chromedriver'
binaryPath = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser' # Ruta de acceso al navegador

options = webdriver.ChromeOptions()
options.binary_location = binaryPath
driver = webdriver.Chrome(executable_path = driverPath, chrome_options = options)

/var/folders/89/g3h9cy012t9b461ggllqnbw00000gn/T/ipykernel_80579/996558949.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = driverPath, chrome_options = options)
/var/folders/89/g3h9cy012t9b461ggllqnbw00000gn/T/ipykernel_80579/996558949.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = driverPath, chrome_options = options)


## **FUNCIÓN PARA EXTRACCIÓN DE LINKS**

In [3]:
def WS_LINKS(búsqueda = list):

    links = []
    títulos = []

    # Función para recolectar links
    def Encontrar_links():
        try:
            # Encontrar títulos de ecuación de búsqueda
            block = driver.find_element(By.CLASS_NAME, "browse2-results") 
            search_list = block.find_elements(By.XPATH, "//h2[contains(@class, 'browse2-result-name')]")
            tipos = block.find_elements(By.XPATH, "//span[@class='browse2-result-type-name']")

            # Recolectar links
            for tipo, search in zip(tipos, search_list):
                if tipo.text == "Conjunto de datos":
                    all_links = search.find_elements(By.TAG_NAME, "a")
                    for a in all_links:
                        if str(a.get_attribute("href")).startswith("https://www.datos.gov.co/") and a.get_attribute("href") not in links: 
                            títulos.append(a.text)
                            links.append(a.get_attribute("href"))
                        else:
                            pass

                # Desplazar hacia abajo para ver todas las búsquedas
                driver.execute_script("arguments[0].scrollIntoView();", search)
        except:
            pass

    print("Links están siendo recolectados")
    
    for i in búsqueda:

        # Introducir ecuación de búsqueda
        e = driver.find_element("xpath", "//input[contains(@id,'autocomplete-search-input-')]")
        e.send_keys(i)

        # Enter
        e.send_keys(Keys.ENTER)

        # Esperar para que la página cargue
        time.sleep(7)

        try:
            # Número de páginas para iterar
            n_pag = driver.find_element(By.XPATH,"//div[@class='pagination']")
            n_pag = n_pag.find_element(By.XPATH, "//a[@aria-label='Última página']").get_attribute('href').rsplit('=', 1)[-1]

            # Recolectar por cada página
            for i in range(int(n_pag)-1):
                Encontrar_links()
                    
                # Ir a la siguiente página
                try:
                    bottom = driver.find_element(By.XPATH, "//*[@aria-label='Página siguiente']")
                    bottom.find_element(By.CLASS_NAME, "icon").click()
                except NoSuchElementException:
                    pass

                time.sleep(7)

        except NoSuchElementException: # Si no hay páginas
            Encontrar_links()
    
    # Crear variable global
    global búsquedas
    
    # Crear dataframe y exportar
    búsquedas = pd.DataFrame(zip(títulos, links),
                             columns = ['Títulos', 'Links'])
    búsquedas.to_csv('links_DA.csv', index = False)
    
    # Imprimir resultados
    print(búsquedas.shape[0])
    return búsquedas

## **FUNCIÓN PARA EXTRACCIÓN DE CONTENIDO**

**Información sobre el conjunto de datos**

* Última actualización de los datos = UltAct_datos
* Última actualización de los meta datos = UltAct_meta
* Fecha de creación = Fecha_creacion
* Suministró los datos = Sumi_datos
* Propietario de conjunto de datos = Propietario

**Información de la Entidad**

* Área o dependencia = Dependencia
* Nombre de la Entidad = Entidad
* Departamento = Departamento
* Municipio = Municipio
* Orden = Orden
* Sector = Sector

**Información de Datos**

* Idioma = Idioma
* Cobertura Geográfica = Cobertura
* Frecuencia de Actualización = Frec_actual
* Fecha Emisión (aaaa-mm-dd) = Emision

**Columnas en el conjunto de datos**
* Nombre de la columna = Nombre
* Descripción = Descripcion
* Tipo = Tipo

In [3]:
def WS_CONTENT(datos):
    # Crear listas vacías para almacenar información
    ID = []
    ## Meta datos 1
    UltAct_datos = []
    UltAct_meta = []
    Fecha_creacion = []
    Sumi_datos = []
    Propietario = []
    ## Meta datos 2
    ### Información de la entidad
    Dependencia = []
    Entidad = []
    Departamento = []
    Municipio = []
    Orden = []
    Sector = []
    ### Información de datos
    Idioma = []
    Cobertura = []
    Frec_actual = []
    Emision = []

    Variables = []

    # Visite cada link uno por uno para hacer scraping a la información
    print('Visitar los links y recopilar información acaba de comenzar.')

    for i in range(len(datos)):
        
        # Abrir link
        try:
            driver.get(datos['Links'][i])
            ID.append(datos['ID'][i])
            time.sleep(7)
            
            try:
                # Dar clic en Mostrar Todo
                driver.find_element(By.XPATH, "//a[@class='table-collapse-toggle more']").click()
                time.sleep(7)
            except NoSuchElementException: # Si no hay más columnas
                pass
            
            # METADATOS
            contents_1 = driver.find_element(By.CLASS_NAME, 'section-content')
            metadata_1 = contents_1.find_element(By.XPATH, "//*[contains(@class, 'metadata-column fancy')]")
            metadata_2 = contents_1.find_element(By.XPATH, "//*[contains(@class, 'metadata-column tables')]")

            # Recolectar información de metadata parte 1
            metadata_1 = metadata_1.find_elements(By.XPATH, "//dd[@class='metadata-detail-group-value']")
            
            ## Guarda los elementos en una sóla lista
            info_1 = [i.text for i in metadata_1]
            ## Divide la lista
            UltAct_datos.append(info_1[0])
            UltAct_meta.append(info_1[1])
            Fecha_creacion.append(info_1[2])
            try:
                Sumi_datos.append(info_1[3])
                Propietario.append(info_1[4])
            except:
                Propietario.append(info_1[3])
            
            # Recolectar información de metadata parte 2
            metadata_2 = metadata_2.find_elements(By.XPATH, "//span[@class='Linkify']")
            ## Guarda los elementos en una sóla lista
            info_2 = [i.text for i in metadata_2]
            ## Divide la lista
            Dependencia.append(info_2[0])
            Entidad.append(info_2[1])
            Departamento.append(info_2[2])
            Municipio.append(info_2[3])
            Orden.append(info_2[4])
            Sector.append(info_2[5])
            Idioma.append(info_2[6])
            Cobertura.append(info_2[7])
            Frec_actual.append(info_2[8])
            Emision.append(info_2[9])
            
            time.sleep(5)

            # Recolectar información de columnas de datos
            contents_2 = driver.find_element(By.XPATH, "//div[@class='table-wrapper']")

            contents_2 = contents_2.find_elements(By.XPATH, "//tr[@class='column-summary']")

            Variables.append([i.text for i in contents_2])

        except:
            print(f'No se puedo realizar scraping en el link {i+1}')
            pass

    # Crear variables globales
    global Info_Datos
    global Info_Variables
        
    # Crear dataframes
    Info_Datos = pd.DataFrame(zip(ID, UltAct_datos, UltAct_meta, Fecha_creacion, Sumi_datos, Propietario, Dependencia, Entidad, Departamento,
                                Municipio, Orden, Sector, Idioma, Cobertura, Frec_actual, Emision),
                            columns = ['ID', 'UltAct_datos', 'UltAct_meta', 'Fecha_creacion', 'Sumi_datos', 'Propietario', 'Dependencia',
                                        'Entidad', 'Departamento', 'Municipio', 'Orden', 'Sector', 'Idioma', 'Cobertura', 'Frec_actual', 'Emision'])

    info_3 = pd.DataFrame(zip(ID, Variables), columns = ['ID', 'Variables'])
    
    Info_Variables = pd.DataFrame(columns = ['ID', 'Nombre', 'Descripcion', 'Tipo'])

    for i in range(len(info_3)):
        for j in info_3["Variables"][i]:
            if len(j.split("\n")) == 3:
                Info_Variables = Info_Variables.append({'ID' : info_3["ID"][i], 'Nombre' : j.split("\n")[0],
                                                        'Descripcion' : j.split("\n")[1], 'Tipo' : j.split("\n")[2]},
                                                       ignore_index = True)
            else:
                Info_Variables = Info_Variables.append({'ID' : info_3["ID"][i], 'Nombre' : j.split("\n")[0],
                                                        'Descripcion' : np.NaN, 'Tipo' : j.split("\n")[1]},
                                                       ignore_index = True)
        

    # Exportar
    Info_Datos.to_csv('Info_Datos.csv', index = False)
    Info_Variables.to_csv('Info_Variables.csv', index = False)
        
    # Imprimir resultados
    print(f'Scraping de los links ha resultado en {Info_Datos.shape[0]} filas de {len(datos)}')

## **EXTRACCIÓN DE LINKS**

In [4]:
# Abrir la página de datos abiertos
driver.get("https://www.datos.gov.co/")

# Esperar para que la página abra
time.sleep(7)

# Ecuaciones de búsqueda
búsqueda = ["Procesos de selección", "Contratista", "Empleo público", "Funcionario", "Servidor", "Mérito", "Carrera administrativa", "Empleados públicos", "Trabajadores oficiales"]

# Extracción links
WS_LINKS(búsqueda)

Links están siendo recolectados
241


,Títulos,Links
0,SECOP II - Procesos de Contratación,https://www.datos.gov.co/Gastos-Gubernamentale...
1,Plan anual de adquisiciones,https://www.datos.gov.co/Minas-y-Energ%C3%ADa/...
2,Plan De Adquisiciones 2016 Alcaldía de Neiva,https://www.datos.gov.co/Econom%C3%ADa-y-Finan...
3,Plan Anual de Adquisiciones - MinEnergía,https://www.datos.gov.co/Econom%C3%ADa-y-Finan...
4,ANM Plan Anual de Adquisiciones Consolidado,https://www.datos.gov.co/Minas-y-Energ%C3%ADa/...
...,...,...
236,BASE DE DATOS EMPRESAS DEL CHOCÓ,https://www.datos.gov.co/Comercio-Industria-y-...
237,Registro Nacional de Turismo - RNT,https://www.datos.gov.co/Comercio-Industria-y-...
238,Bases de datos sobre el número de mujeres que ...,https://www.datos.gov.co/dataset/Bases-de-dato...
239,INTEGRA (Índice Integral de Legalidad),https://www.datos.gov.co/Funci%C3%B3n-p%C3%BAb...


## **EXTRACCIÓN DE CONTENIDO**

In [4]:
# importar datos extraídos anteriormente
datos = pd.read_csv("./links_DA.csv")

# Generar ids
datos['ID'] = datos.index + 1

# Reorganizar las columnas
datos = datos[["ID", "Títulos", "Links"]]

datos

,ID,Títulos,Links
0,1,SECOP II - Procesos de Contratación,https://www.datos.gov.co/Gastos-Gubernamentale...
1,2,Plan anual de adquisiciones,https://www.datos.gov.co/Minas-y-Energ%C3%ADa/...
2,3,Plan De Adquisiciones 2016 Alcaldía de Neiva,https://www.datos.gov.co/Econom%C3%ADa-y-Finan...
3,4,Plan Anual de Adquisiciones - MinEnergía,https://www.datos.gov.co/Econom%C3%ADa-y-Finan...
4,5,ANM Plan Anual de Adquisiciones Consolidado,https://www.datos.gov.co/Minas-y-Energ%C3%ADa/...
...,...,...,...
236,237,BASE DE DATOS EMPRESAS DEL CHOCÓ,https://www.datos.gov.co/Comercio-Industria-y-...
237,238,Registro Nacional de Turismo - RNT,https://www.datos.gov.co/Comercio-Industria-y-...
238,239,Bases de datos sobre el número de mujeres que ...,https://www.datos.gov.co/dataset/Bases-de-dato...
239,240,INTEGRA (Índice Integral de Legalidad),https://www.datos.gov.co/Funci%C3%B3n-p%C3%BAb...


In [5]:
WS_CONTENT(datos)

Visitar los links y recopilar información acaba de comenzar.
Scraping de los links ha resultado en 241 filas de 241


In [6]:
Info_Datos

,ID,UltAct_datos,UltAct_meta,Fecha_creacion,Sumi_datos,Propietario,Dependencia,Entidad,Departamento,Municipio,Orden,Sector,Idioma,Cobertura,Frec_actual,Emision
0,1,28 de diciembre de 2022,8 de julio de 2022,30 de septiembre de 2019,Colombia Compra Eficiente,Colombia Compra Eficiente,Subdirección de IDT,Agencia Nacional de Contratación Pública Colom...,Bogotá D.C.,Bogotá D.C.,Nacional,Planeación,Español,Nacional,Diaria,2019-10-01
1,2,24 de septiembre de 2020,15 de julio de 2022,17 de mayo de 2017,Comisión de Regulación de Energía y Gas,Comisión de Regulación de Energía y Gas,Mauricio Velosa,Comisión de Regulación de Energía y Gas,Bogotá D.C.,Bogotá D.C.,Nacional,Minas y Energía,Español,Nacional,Anual,2017-05-17
2,3,10 de agosto de 2016,14 de octubre de 2020,10 de agosto de 2016,Alcaldía Municipal de Neiva en Huila,Secretaria TIC,ALCALDIA MUNICIPAL DE NEIVA,Alcaldía Municipal de Neiva,Huila,Neiva,Territorial,Función Pública,Español,Municipal,Anual,2016-02-05
3,4,13 de mayo de 2019,15 de julio de 2022,13 de octubre de 2016,MINISTERIO DE MINAS Y ENERGÍA,Ministerio de Minas y Energia,MinMinas,Ministerio de Minas y Energía,Bogotá D.C.,Bogotá D.C.,Nacional,Minas y Energía,Español,Nacional,Anual,2016-10-13
4,5,16 de diciembre de 2022,16 de diciembre de 2022,11 de mayo de 2018,Agencia Nacional de Minería ANM,ANM Agencia Nacional de Minería Contratos,Grupo de Contratos,Agencia Nacional de Minería,Bogotá D.C.,Bogotá D.C.,Nacional,Minas y Energía,Español,Nacional,Mensual,2018-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,237,9 de diciembre de 2020,9 de diciembre de 2020,25 de septiembre de 2020,CAMARA DE COMERCIO DEL CHOCÓ,YILBER PALACIOS,COMERCIO,Cámara de Comercio del Chocó,Chocó,Quibdó,Territorial,"Comercio, Industria y Turismo",Español,Departamental,Anual,2020-09-24
237,238,20 de diciembre de 2022,20 de diciembre de 2022,13 de abril de 2021,"Ministerio de Comercio, Industria y Turismo",MINCIT,Dirección de Análisis Sectorial,"Ministerio de Comercio, Industria y Turismo",Bogotá D.C.,Bogotá D.C.,Nacional,"Comercio, Industria y Turismo",Español,Nacional,Semestral,2021-06-30
238,239,27 de agosto de 2021,15 de julio de 2022,27 de agosto de 2021,Cámara de Comercio de Cartagena,Cámara de Comercio de Cartagena,Servicios Empresariales,Cámara de Comercio de Cartagena,Bolívar,Cartagena,Territorial,"Comercio, Industria y Turismo",Español,Municipal,Trienio,27/08/2021
239,240,24 de julio de 2020,15 de julio de 2022,26 de mayo de 2020,Procuraduría General de la Nación,datosabiertos@procuraduria.gov.co,Delegada para la Defensa del Patrimonio Públic...,Procuraduría General de la Nación,Bogotá D.C.,Bogotá D.C.,Nacional,Organismos de control y vigilancia,Español,Nacional,Anual,2019-10-30


In [7]:
Info_Variables

,ID,Nombre,Descripcion,Tipo
0,1,Entidad,Nombre de la Entidad que publica el proceso de...,Texto simple
1,1,Nit Entidad,NIT de la Entidad que publicó el proceso,Texto simple
2,1,Departamento Entidad,Departamento en el cual está registrada la ent...,Texto simple
3,1,Ciudad Entidad,Ciudad en la cual está registrada la entidad,Texto simple
4,1,OrdenEntidad,"Orden de la Entidad (Nacional, Regional)",Texto simple
...,...,...,...,...
4470,241,EntidadKAutorizoNroID,Identificación de la entidad que realizó la va...,Texto simple
4471,241,EntidadKAutorizoNombre,Nombre de la entidad que autorizó la valoración,Texto simple
4472,241,DESCRIP_PERFILFUNCIONAMIENTO_1,Explicación de componente del perfil de funcio...,Texto simple
4473,241,DESCRIP_PERFILFUNCIONAMIENTO_2,Explicación de Componente del perfil de funcio...,Texto simple
